# Send REST requests to the WeatherApp.API running in Aspire

In [4]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [5]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-495769a054acd4aff0700d77874ab912-0a0197256ebf4dd3-00
Name,Value
Date,"Wed, 14 May 2025 10:12:41 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌦️🌈🔆

In [6]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-a5adaa65c369f3a09bdc34523b660c51-98a5075d21eb061a-00
Name,Value
Date,"Wed, 14 May 2025 10:12:46 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-05-14T10:12:47.0457139Z"""
requestId,"""7515550e-b3d1-42e9-a416-e9d7e06bc1b6"""


## POST Colected Weather Data 🌡️☔💨

In [2]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

watermelon-13

In [13]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
X-Request-ID,191b9a5e-290e-4e43-808d-9f37d3a8ef88
traceparent,00-11c03435fdd246fc53c2e5eabbdcf83e-27bffdb72984e7fe-00
Content-Type,application/json
Content-Length,278
Name,Value
Date,"Wed, 14 May 2025 11:05:06 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,80


<div style="display:block; height:40rem;"></div>

## GET Traces 🤖

In [15]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("http://localhost:8000"), DefaultRequestHeaders = { { "X-Api-Key", "123456789" } } };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 1 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 796175db61c2c941 Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 1 1 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 7d2b9a328d012603 Tags key value domain-event.streamId "53ad5f16-dc02-426b-9735-a8dc0c9050b3" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 2 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 2 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId b5624fb37edee484 Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 2 3 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 4b8e27cbaf61b808 Tags key value domain-event.streamId "53ad5f16-dc02-426b-9735-a8dc0c9050b3" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 4 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId d23f90f3a081e6f3 Tags key value domain-event.streamId "53ad5f16-dc02-426b-9735-a8dc0c9050b3" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 5 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 3c91a55420a3315c Tags key value domain-event.streamId "53ad5f16-dc02-426b-9735-a8dc0c9050b3" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 6 NotificationService-----------> User Notication Sent----------> ✉️ reference: watermelon-13 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 72913a6fef261a57 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "watermelon-13" user-notification-event.timestamp "2025-05-14T11:05:17.8553422+00:00" 7 NotificationService-----------> User Notication Sent----------> ✉️ reference: watermelon-13 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId 9769f22d29e1ca8a Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "watermelon-13" user-notification-event.timestamp "2025-05-14T11:05:13.0913002+00:00" 8 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e SpanId d602ab765e993987 Tags key value domain-event.streamId "53ad5f16-dc02-426b-9735-a8dc0c9050b3" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 9 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 11c03435fdd246fc53c2e5eabbdcf83e